In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import sys
import esig
import numpy as np
import random
import matplotlib.pyplot as plt
import sklearn.metrics
import itertools
import os.path
import time
import zipfile
import iisignature
from joblib import Memory, Parallel, delayed
from datetime import datetime
from signal import signal
from tqdm import tqdm
from functools import lru_cache
from scipy.stats import chi2
from sklearn.metrics import roc_auc_score
from sklearn.covariance import EmpiricalCovariance, MinCovDet
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import normalize
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import SpectralClustering

sys.path.insert(0, 'C:\\Users\\dmitr\\Desktop\\project work')

from signature_calculation import calculate_signatures
from path_calculation import paths_from_edges
from path_calculation import paths_from_vertices
from interleave_clusters import interleave

import outlier_calc
import clustering

from variance import variance

day = 86400

0.6444444444444445


In [3]:
clean_events = pd.read_csv('C:\\Users\\dmitr\\Desktop\\project work\\dummy data\\clean_data.csv')
anomalous_events_labels = pd.read_csv('C:\\Users\\dmitr\\Desktop\\project work\\dummy data\\data_anomaly_labels.csv')
anomalous_events_timestamps = set(anomalous_events_labels['id'].values.tolist())
events = pd.read_csv('C:\\Users\\dmitr\\Desktop\\project work\\dummy data\\data.csv')

In [12]:
cluster_1 = pd.read_csv('C:\\Users\\dmitr\\Desktop\\project work\\dummy data\\data_c1.csv')
cluster_2 = pd.read_csv('C:\\Users\\dmitr\\Desktop\\project work\\dummy data\\data_c2.csv')

In [4]:
class hyperparameters:
    normalising_data = False 
    scale_data = True
    preprocess_data_columnwise = False
    normalising_signature = False
    scale_signature = False
    preprocess_signature_columnwise = False
    covariance_method = "Emperical"
    knn_metric = 'mahalanobis'
    observed = [np.int_(0), np.int_(1), np.int_(2), np.int_(3), np.int_(4)]
    depth = 2
    time_variable = True
    daytime_variable = True
    stream_length = 7 * day
    stream_spacing = 2 * day

    no_extrapollation = True
    lead_lag = False

    neighbours = 5
 

In [5]:
(corpus, _) = paths_from_edges(clean_events, set([]), hyperparameters)
(streams, outlier_count) = paths_from_edges(events, anomalous_events_timestamps, hyperparameters)
labels = [outlier_count[i] > 0 for i in range(len(outlier_count))]

In [6]:
streams_signatures = calculate_signatures(streams, hyperparameters)
corpus_signatures = calculate_signatures(corpus, hyperparameters)

In [8]:
pred = outlier_calc.knn(streams_signatures, hyperparameters)
tpr_knn_edges, fpr_knn_edges, area_knn_edges = outlier_calc.evaluate(pred, labels)
print(area_knn_edges)

0.8781049679487178


In [9]:
certainty = outlier_calc.mah_distance(streams_signatures, hyperparameters)
tpr_mah_edges, fpr_mah_edges, area_mah_edges = outlier_calc.evaluate(certainty, labels)
print(area_mah_edges)

0.8491586538461537


In [10]:
score = outlier_calc.isolation_forest(corpus_signatures, streams_signatures, hyperparameters)
tpr_iso_edges, fpr_iso_edges, area_iso_edges = outlier_calc.evaluate(score, labels)
print(area_iso_edges)

0.7700320512820513


In [11]:
score_svm_unsup = outlier_calc.one_class_SVM_unsupervised(streams_signatures, hyperparameters)
tpr_svmus_edges, fpr_svmus_edges, area_svmus_edges = outlier_calc.evaluate(score_svm_unsup, labels)
print(area_svmus_edges)

0.47896634615384615


In [14]:
res = variance(streams, corpus, hyperparameters.depth)
tpr_conformance_edges, fpr_conformance_edges, area_conformance_edges = outlier_calc.evaluate(res, labels)
print(area_conformance_edges)

Computing variances: 100%|███████████████████████████████████████████████████████████| 397/397 [00:24<00:00, 16.25it/s]

0.9242788461538461


In [13]:
(c1_streams, _) = paths_from_edges(cluster_1, set([]), hyperparameters)
(c2_streams, _) = paths_from_edges(cluster_2, set([]), hyperparameters)

In [14]:
c1_signatures = calculate_signatures(c1_streams, hyperparameters)
c2_signatures = calculate_signatures(c2_streams, hyperparameters)

In [15]:
clustering_streams, clustering_labels = interleave(c1_signatures, c2_signatures)

In [17]:
pred_labels_spectral_vertices = clustering.spectral_clustering(clustering_streams)
print(clustering.evaluate(pred_labels_spectral_vertices, clustering_labels))

C:\Users\dmitr\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\manifold\_spectral_embedding.py:259: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(


0.49948717948717947


In [18]:
pred_labels_kmeans_edges = clustering.kmeans(clustering_streams)
print(clustering.evaluate(pred_labels_kmeans_edges, clustering_labels))

0.49948717948717947


In [19]:
pred_labels_optics_edges = clustering.optics(clustering_streams)
print(clustering.evaluate(pred_labels_optics_edges, clustering_labels))

0.4995039932744851


In [ ]:
(corpus2, _) = paths_from_vertices(clean_events, set([]), hyperparameters)
(streams2, outlier_count2) = paths_from_vertices(events, anomalous_events_timestamps, hyperparameters)
labels2 = [outlier_count[i] > 0 for i in range(len(outlier_count))]

In [ ]:
streams_signatures2 = calculate_signatures(streams2, hyperparameters)
corpus_signatures2 = calculate_signatures(corpus2, hyperparameters)

In [ ]:
pred2 = outlier_calc.knn(streams_signatures2)
tpr_knn_vertices, fpr_knn_vertices, area_knn_vertices = outlier_calc.evaluate(pred2, labels2)
print(area_knn_vertices)

In [ ]:
certainty2 = outlier_calc.mah_distance(streams_signatures2, hyperparameters)
tpr_mah_vertices, fpr_mah_vertices, area_mah_vertices = outlier_calc.evaluate(certainty2, labels2)
print(area_mah_vertices)

In [ ]:
score2 = outlier_calc.isolation_forest(corpus_signatures2, streams_signatures2, hyperparameters)
tpr_iso_vertices, fpr_isovertices, area_iso_vertices = outlier_calc.evaluate(score2, labels)
print(area_mah_vertices)

In [ ]:
score_svm_unsup2 = outlier_calc.one_class_SVM_unsupervised(streams_signatures2, hyperparameters)
tpr_svmus_vertices, fpr_svmus_vertices, area_svmus_vertices = outlier_calc.evaluate(score_svm_unsup2, labels)
print(area_svmus_vertices)

In [23]:
res = variance(streams2, corpus2, hyperparameters.depth)
tpr_conformance_vertices, fpr_conformance_vertices, area_conformance_vertices = outlier_calc.evaluate(res, labels)
print(area_conformance_vertices)

Computing variances: 100%|███████████████████████████████████████████████████████████| 397/397 [00:24<00:00, 16.14it/s]

0.8048878205128205


In [ ]:
(c1_streams2, _) = paths_from_vertices(cluster_1, set([]), hyperparameters)
(c2_streams2, _) = paths_from_vertices(cluster_2, set([]), hyperparameters)

In [ ]:
c1_signatures2 = calculate_signatures(c1_streams2, hyperparameters)
c2_signatures2 = calculate_signatures(c2_streams2, hyperparameters)

In [ ]:
clustering_streams2, clustering_labels2 = interleave(c1_signatures2, c2_signatures2)

In [ ]:
pred_labels_spectral_vertices = clustering.spectral_clustering(clustering_streams2)
print(clustering.evaluate(pred_labels_spectral_vertices, clustering_labels2))

In [ ]:
pred_labels_kmeans_vertices = clustering.kmeans(clustering_streams2)
print(clustering.evaluate(pred_labels_kmeans_vertices, clustering_labels2))

In [ ]:
pred_labels_optics_vertices = clustering.optics(clustering_streams2)
print(clustering.evaluate(pred_labels_optics_vertices, clustering_labels2))